In [5]:
import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pickle
from concurrent.futures import ThreadPoolExecutor
from IPython.display import clear_output
import pathlib
import time
import keyboard

In [6]:
def refresh_cookies(email, pw):
    url = f'https://kktix.com'
    options = uc.ChromeOptions()
    options.add_argument("--headless")
    driver = uc.Chrome(options=options)
    driver.get(url)
    WebDriverWait(driver, 5).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "li[class='not-signed-in']")))[1].click()
    WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.CSS_SELECTOR, "input[id='user_login']"))).send_keys(email)
    WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.CSS_SELECTOR, "input[id='user_password']"))).send_keys(pw)
    WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.CSS_SELECTOR, "[class='btn btn normal btn-login']"))).click()
    time.sleep(2)
    pickle.dump(driver.get_cookies(), open("cookies.pkl", "wb"))
    driver.quit()
def remove_twd_comma(s):
    return int(s.replace("TWD$", "").replace(",", ""))

In [7]:
price_range = [1200, 500]
name_keyword = ["B", "黃3I"]
ticket_quantity = 2
cutoff_price = 8000
cutoff_ticket_quantity = 1
# with open("./cred.txt", "r") as f:
#     email, pw = f.read().split("\n")
# refresh_cookies(email, pw)

In [8]:
events = {
    "": "ruspiano2024",
    "10/4": "a8249618-00a1c",
    "10/5": "a8249618-01afw",
    "10/6": "a8249618-02bew",
}
event_id = events[""]
url = f'https://kktix.com/events/{event_id}/registrations/new'
options = uc.ChromeOptions()
# prefs = {"profile.managed_default_content_settings.images": 2}
# options.add_experimental_option("prefs", prefs)
# options.add_argument('--blink-settings=imagesEnabled=false')
# options.add_argument(f"--load-extension={pathlib.Path().absolute()}/CapSolver/")
options.page_load_strategy = 'normal'
options.add_argument(f"--load-extension={pathlib.Path().absolute()}/ublock/")
driver = uc.Chrome(options=options)
with ThreadPoolExecutor() as executor:
    executor.submit(lambda: driver.set_window_position(2000, 0))
    executor.submit(lambda: driver.maximize_window())
    executor.submit(lambda: driver.get(url))
cookies = pickle.load(open("cookies.pkl", "rb"))
for cookie in cookies:
    driver.add_cookie(cookie)
driver.refresh()

In [35]:
found_type = False
while(True):

    clear_output(wait=True)
    if(keyboard.is_pressed('q')): 
        break

    if(found_type):
        try:
            driver.switch_to.alert.accept()
            found_type = False
        except:
            continue

    with ThreadPoolExecutor() as executor:
        executor.submit(lambda: WebDriverWait(driver, 2).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "input[id='person_agree_terms']"))).click())
        ticket_type_collection = executor.submit(lambda: WebDriverWait(driver, 2).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "div[ng-controller='TicketCtrl']")))).result()
        manual_automatic_button = executor.submit(lambda: WebDriverWait(driver, 2).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "div[ng-show='anyPurchasable()'] > button")))).result()

    for price in price_range:
        # print(f"Currently finding ${price} tickets:")
        for ticket_type in ticket_type_collection:
            type_price = remove_twd_comma(ticket_type.find_element(By.CSS_SELECTOR, "[class='ng-binding ng-scope']").text)
            type_name = ticket_type.find_element(By.CSS_SELECTOR, "span[class='ticket-name ng-binding']").text.replace("\n", " ")
            if(type_price == price):
                try:
                    status = ticket_type.find_element(By.CSS_SELECTOR, "[class='ticket-quantity ng-binding ng-scope']").text
                    # print(f"* {type_name} is currently {status}")
                    continue
                except:
                    pass
                try:
                    tickets_left = ticket_type.find_element(By.CSS_SELECTOR, "span[ng-if='showCapacityNotice']").text
                    if(int(tickets_left.rstrip(" Left")) < ticket_quantity):
                        # print(tickets_left)
                        continue
                except:
                    pass
                # ticket_quantity = ticket_quantity if price < cutoff_price else cutoff_ticket_quantity
                input_field = ticket_type.find_element(By.CSS_SELECTOR,  "input")
                input_field.clear()
                input_field.send_keys(ticket_quantity)
                manual_automatic_button[-1].click()
                found_type = True
                print(f"* Found {type_name} ${price} tickets available")
                break
        if(found_type):
            break
        else:
            pass
            # print(f"* Did not find ${price} tickets")
    if(not found_type):
        # print("! Did not find any ticket type available, refreshing...")
        driver.refresh()

* Found $1,200 全 $1200 tickets available
